### Star Ratings Results
- Input from AimBig notebook, stat_regression 
- 6 steps of the star rating results
- GroupBy object = (Contract ID, ESA Code, Speciality_site_type_code)

In [1272]:
import datetime as dt
import re
import json
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, f1_score
from datetime import date, timedelta
from datetime import datetime
import warnings
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sn
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# reading in file with numerator/denominator for each participant, performance measure against contract
df_merged = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_final.csv')
# creating groupBy object 
df_merged = df_merged.drop_duplicates()
#df_group = df_merged.groupby(['CONTRACT_ID', 'ESA_CODE', 'SPECIALIST_SITE_TYPE_CODE'])

df_merged.columns

df_merged = df_merged.drop(columns=['Unnamed: 0'])

In [1273]:
df_merged

,Unnamed: 0.1,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,Num_13_full_exp,Num_26_full_exp,Num_26_path_exp,Num_26_wrkast_exp,Num_52_full_exp,Num_52_path_exp
0,0,8653691003,NaN,NaN,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,7870290004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,6498977509,NaN,NaN,AimBig Employment LEICHHARDT,0212845J,AALL,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,3,6638331004,NaN,NaN,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2792458409,NaN,NaN,AimBig Employment ROCKINGHAM,0212831C,MENH,1.0,1.0,0.0,...,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,1104,2836891309,NaN,NaN,AimBig Employment CORIO,0212862K,AALL,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1105,1105,8771045004,NaN,NaN,AimBig Employment FRANKSTON,0212818F,MENH,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1106,1106,15325790,NaN,NaN,AimBig Employment MACKAY,0212803K,MUSK,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1107,1107,690092007,NaN,NaN,AimBig Employment Pty Ltd SUNBURY,0212817E,AALL,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Step 1) Calculating Actual Performance
#### Actual Performance
groupBy key = (Contract ID, ESA Code, Speciality_site_type_code)

##### Pre-Quarter

- Count NaNs -> place as 0

In [1275]:
df_merged = df_merged.fillna(value = 0)
df_merged

,Unnamed: 0.1,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,Num_13_full_exp,Num_26_full_exp,Num_26_path_exp,Num_26_wrkast_exp,Num_52_full_exp,Num_52_path_exp
0,0,8653691003,0.0,0.0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,7870290004,0.0,0.0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,6498977509,0.0,0.0,AimBig Employment LEICHHARDT,0212845J,AALL,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,3,6638331004,0.0,0.0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2792458409,0.0,0.0,AimBig Employment ROCKINGHAM,0212831C,MENH,1.0,1.0,0.0,...,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1104,1104,2836891309,0.0,0.0,AimBig Employment CORIO,0212862K,AALL,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1105,1105,8771045004,0.0,0.0,AimBig Employment FRANKSTON,0212818F,MENH,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1106,1106,15325790,0.0,0.0,AimBig Employment MACKAY,0212803K,MUSK,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1107,1107,690092007,0.0,0.0,AimBig Employment Pty Ltd SUNBURY,0212817E,AALL,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [1277]:
df_merged = df_merged.drop(columns=['Num_13', 'Den_13'])

### Aggregating to Site Level 

In [1278]:
df_group = df_merged.groupby(['Contract_ID', 'Site_Name',  'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_26_full', 'Den_26_full',
       'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast',
       'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()

df_group['actual_13_full'] = 0
df_group['actual_26_full'] = 0 
df_group['actual_26_path'] = 0
df_group['actual_26_wrkast'] = 0
df_group['actual_52_full'] = 0
df_group['actual_52_path'] = 0

df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100




Renaming df_group -> df_pre_actual

In [1279]:
df_pre_actual = df_group
df_pre_actual.to_csv('C:/Users/AManalo/star_ratings_new/Output Files/test_sites.csv')

##### Post-Quarter

In [1280]:
# reading in the merged file
df_merge_post = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_merge_post.csv')
df_merge_post = df_merge_post.fillna(value = 0)
df_merge_post = df_merge_post.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_CODE' : 'Site_Code', 'SITE_DESCRIPTION': 'Site_Name', 'SPECIALIST_SITE_TYPE_CODE' : 'specialist_site_type_code'})
df_merge_post.head(5)

,Unnamed: 0.1,JOB_SEEKER_ID,Num_13,Den_13,Site_Name,Contract_ID,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,Num_13_full_exp,Num_26_full_exp,Num_26_path_exp,Num_26_wrkast_exp,Num_52_full_exp,Num_52_path_exp
0,0,8653691003,0.0,0.0,AimBig Employment FAIRFIELD,0212899A,AALL,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,7870290004,0.0,0.0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,6498977509,0.0,0.0,AimBig Employment LEICHHARDT,0212845J,AALL,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,3,6638331004,0.0,0.0,AimBig Employment NORTH LAKES,0212795D,AALL,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2792458409,0.0,0.0,AimBig Employment ROCKINGHAM,0212831C,MENH,1.0,1.0,0.0,...,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [1281]:
df_merge_post = df_merge_post.drop(columns=['Num_13', 'Den_13'])

In [1282]:
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code', 'JOB_SEEKER_ID'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()
df_group = df_merge_post.groupby(['Contract_ID', 'Site_Name', 'specialist_site_type_code'])['Num_13_full', 'Den_13_full', 'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path', 'Num_26_wrkast', 'Den_26_wrkast', 'Num_52_full', 'Den_52_full', 'Num_52_path', 'Den_52_path'].sum()


#df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_post.csv')

**Calculating Actual Performance**

In [1284]:

df_group['actual_13_full'] = df_group['Num_13_full']/df_group['Den_13_full'] * 100
df_group['actual_26_full'] = df_group['Num_26_full']/df_group['Den_26_full'] * 100

df_group['actual_26_path'] = df_group['Num_26_path']/df_group['Den_26_path'] * 100
df_group['actual_26_wrkast'] = df_group['Num_26_wrkast']/df_group['Den_26_wrkast'] * 100
df_group['actual_52_full'] = df_group['Num_52_full']/df_group['Den_52_full'] * 100
df_group['actual_52_path'] = df_group['Num_52_path']/df_group['Den_52_path'] * 100

df_group = df_group.fillna(0.5)

df_group.to_csv('C:/Users/AManalo/star_ratings_new/df_group_test.csv')

## Step 2: Expected Performance

Using statistical regression, which takes account of variable Participant and labour market characteristics and the number of Participants in the denominator post-quarterisation, the Star Ratings model calculates the number of outcomes that the Contract could be expected to achieve.

In [1286]:
df_merge_post.columns

Index(['Unnamed: 0.1', 'JOB_SEEKER_ID', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_13_full', 'Den_13_full',
       'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path',
       'Num_26_wrkast', 'Den_26_wrkast', 'Unnamed: 0', 'Num_52_full',
       'Den_52_full', 'Num_52_path', 'Den_52_path', 'Num_13_full_exp',
       'Num_26_full_exp', 'Num_26_path_exp', 'Num_26_wrkast_exp',
       'Num_52_full_exp', 'Num_52_path_exp'],
      dtype='object')

In [1287]:
# Get expected columns from df_merged
df_merge_post.reset_index()
df_merge_post.columns

df_exp = df_merge_post[['JOB_SEEKER_ID', 'Contract_ID',  'Site_Name', 'specialist_site_type_code', 'Num_13_full_exp', 'Num_26_full_exp', 'Num_26_path_exp', 'Num_26_wrkast_exp', 'Num_52_full_exp', 'Num_52_path_exp']]
       

Merging df_exp with the denominator from post quarterisation outcomes

In [1288]:
df_merge_post.columns

Index(['Unnamed: 0.1', 'JOB_SEEKER_ID', 'Site_Name', 'Contract_ID',
       'specialist_site_type_code', 'Num_13_full', 'Den_13_full',
       'Num_26_full', 'Den_26_full', 'Num_26_path', 'Den_26_path',
       'Num_26_wrkast', 'Den_26_wrkast', 'Unnamed: 0', 'Num_52_full',
       'Den_52_full', 'Num_52_path', 'Den_52_path', 'Num_13_full_exp',
       'Num_26_full_exp', 'Num_26_path_exp', 'Num_26_wrkast_exp',
       'Num_52_full_exp', 'Num_52_path_exp'],
      dtype='object')

In [1289]:
df_exp.to_csv('C:/Users/AManalo/star_ratings_new/df_exp.csv')

In [1290]:
df_merge_post
df_post_den = df_merge_post[["JOB_SEEKER_ID", "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path"]]
df_exp_per = df_exp.merge(df_post_den, on=['JOB_SEEKER_ID'], how='inner')


df_exp_per = df_exp_per.groupby(['Contract_ID', 'Site_Name',  'specialist_site_type_code'])[ "Den_13_full", "Den_26_full", "Den_26_path", "Den_26_wrkast", "Den_52_full", "Den_52_path", 'Num_13_full_exp', 'Num_26_full_exp', 'Num_52_full_exp', 'Num_52_path_exp', 'Num_26_wrkast_exp', 'Num_26_path_exp'].sum()



Calculating expected performance rate
- for each performance measure

In [1291]:
df_exp_per['exPer_13_full'] = df_exp_per['Num_13_full_exp'] / df_exp_per['Den_13_full'] * 100
df_exp_per['exPer_26_full'] = df_exp_per['Num_26_full_exp'] / df_exp_per['Den_26_full'] * 100
df_exp_per['exPer_26_path'] = df_exp_per['Num_26_path_exp'] / df_exp_per['Den_26_path'] * 100
df_exp_per['exPer_26_wrkast'] = df_exp_per['Num_26_wrkast_exp'] / df_exp_per['Den_26_wrkast'] * 100
df_exp_per['exPer_52_full'] = df_exp_per['Num_52_full_exp'] / df_exp_per['Den_52_full'] * 100
df_exp_per['exPer_52_path'] = df_exp_per['Num_52_path_exp'] / df_exp_per['Den_52_path'] * 100


df_exp_per['exPer_26_path'] = np.where(df_exp_per['exPer_26_path'] == 0, 0.5, df_exp_per['exPer_26_path'])
df_exp_per['exPer_52_path'] = np.where(df_exp_per['exPer_52_path'] == 0, 0.5, df_exp_per['exPer_52_path'])
df_exp_per['exPer_26_wrkast'] = np.where(df_exp_per['exPer_26_wrkast'] == 0, 0.5, df_exp_per['exPer_26_wrkast'])
df_exp_per = df_exp_per.replace([0, np.nan, np.inf, -np.inf], 0.5)

In [1292]:
df_exp_per.head(10)

Den_13_full  \
Contract_ID Site_Name                         specialist_site_type_code                
0212790K    AimBig Employment NORWOOD         MUSK                               1.0   
            AimBig Employment PROSPECT        MUSK                               5.0   
0212791K    AimBig Employment MILE END        MENH                              19.0   
            AimBig Employment Pty Ltd. SEATON MENH                               7.0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                              19.0   
...                                                                              ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                              14.0   
            AimBig Employment INGLEBURN       AALL                              12.0   
0212904D    AimBig Employment HURSTVILLE      AALL                              18.0   
            AimBig Employment MIRANDA         AALL                              16.0   
            AimBig Employment ROCKDALE        AALL                              16.0   

                                                                         Den_26_full  \
Contract_ID Site_Name                         specialist_site_type_code                
0212790K    AimBig Employment NORWOOD         MUSK                               1.0   
            AimBig Employment PROSPECT        MUSK                               1.0   
0212791K    AimBig Employment MILE END        MENH                              29.0   
            AimBig Employment Pty Ltd. SEATON MENH                               1.0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                              20.0   
...                                                                              ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                              27.0   
            AimBig Employment INGLEBURN       AALL                               9.0   
0212904D    AimBig Employment HURSTVILLE      AALL                              26.0   
            AimBig Employment MIRANDA         AALL                              17.0   
            AimBig Employment ROCKDALE        AALL                               0.5   

                                                                         Den_26_path  \
Contract_ID Site_Name                         specialist_site_type_code                
0212790K    AimBig Employment NORWOOD         MUSK                               0.5   
            AimBig Employment PROSPECT        MUSK                               0.5   
0212791K    AimBig Employment MILE END        MENH                              11.0   
            AimBig Employment Pty Ltd. SEATON MENH                               8.0   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                               2.0   
...                                                                              ...   
0212902B    AimBig Employment CAMPBELLTOWN    AALL                               3.0   
            AimBig Employment INGLEBURN       AALL                               1.0   
0212904D    AimBig Employment HURSTVILLE      AALL                               3.0   
            AimBig Employment MIRANDA         AALL                               2.0   
            AimBig Employment ROCKDALE        AALL                               0.5   

                                                                         Den_26_wrkast  \
Contract_ID Site_Name                         specialist_site_type_code                  
0212790K    AimBig Employment NORWOOD         MUSK                                 0.5   
            AimBig Employment PROSPECT        MUSK                                 0.5   
0212791K    AimBig Employment MILE END        MENH                                 0.5   
            AimBig Employment Pty Ltd. SEATON MENH                                 0.5   
0212793B    AimBig Employment BURLEIGH HEADS  MENH                                 0.5   
...     

Read in the Sub368 file and add merge this with the df_exp_per
- The sub368 averages end in column '_avg'

In [1293]:
df_sub = pd.read_csv('C:/Users/AManalo/star_ratings_new/df_sub.csv')
df_sub = df_sub.groupby(['CONTRACT_ID', 'SITE_CODE', 'SPECIALTY_CD', 'SITE_NAME'])['52_full_avg', '52_path_avg',  '26_wrkast_avg', '26_full_avg', '26_path_avg', '13_full_avg'].sum()
df_sub = df_sub.reset_index()
df_sub = df_sub.rename(columns={'CONTRACT_ID' : 'Contract_ID', 'SITE_NAME' : 'Site_Name', 'SPECIALTY_CD' : 'specialist_site_type_code'})

In [1294]:
df_sub

,Contract_ID,SITE_CODE,specialist_site_type_code,Site_Name,52_full_avg,52_path_avg,26_wrkast_avg,26_full_avg,26_path_avg,13_full_avg
0,0212789F,FV18,MENH,AimBig Employment DULWICH,15.7,1.1,71.9,22.8,1.2,24.0
1,0212789F,FV28,MENH,AimBig Employment MAYLANDS,15.7,1.1,71.9,22.8,1.2,24.0
2,0212789F,MO87,MENH,AimBig Employment PROSPECT,15.7,1.1,71.9,22.8,1.2,24.0
3,0212790K,FV18,MUSK,AimBig Employment DULWICH,15.7,1.1,71.9,22.8,1.2,24.0
4,0212790K,FV28,MUSK,AimBig Employment MAYLANDS,15.7,1.1,71.9,22.8,1.2,24.0
...,...,...,...,...,...,...,...,...,...,...
256,0212903C,FX68,AALL,Aimbig Employment RICHMOND,17.2,1.3,68.4,21.5,1.1,22.4
257,0212903C,FX78,AALL,Aimbig Employment WINDSOR,17.2,1.3,68.4,21.5,1.1,22.4
258,0212904D,A981,AALL,AimBig Employment HURSTVILLE,17.2,1.3,68.4,21.5,1.1,22.4
259,0212904D,A991,AALL,AimBig Employment MIRANDA,17.2,1.3,68.4,21.5,1.1,22.4


Merge the df_sub columns with the df_exp_per

In [1295]:
df_exp_per = df_exp_per.reset_index()
df_exp_per = df_exp_per.merge(df_sub, on=['Site_Name', 'Contract_ID', 'specialist_site_type_code'], how='inner')


## Step 3: Calculate the Performance Measure Scores (Ratio of Actual Outcomes : Expected Outcomes)

Combined 13, 25, 52 week scores are calculated based on the weightings of each sub type measure. 
Actual rate: actual rate (post-quarter)
Expected rate: expected performance rate


To-Fix: Need to adjust the weights for each performance measure IF the denominator in the pre-quarter outcomes are 0 or very small. 

- ESS: min 20 participants (combined) in the pre-quarterisation denominators for the following: 13-week full outcomes (min 5 participants), ongoing support  
- DMS: min 20 participants in the pre-quarterisation denominator for 13 week full outcomes measure

In [1296]:
df_group.reset_index()

,Contract_ID,Site_Name,specialist_site_type_code,Num_13_full,Den_13_full,Num_26_full,Den_26_full,Num_26_path,Den_26_path,Num_26_wrkast,...,Num_52_full,Den_52_full,Num_52_path,Den_52_path,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path
0,0212790K,AimBig Employment NORWOOD,MUSK,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.000000,100.000000,0.500000,0.5,0.500000,0.5
1,0212790K,AimBig Employment PROSPECT,MUSK,2.0,5.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,40.000000,100.000000,0.500000,0.5,100.000000,0.5
2,0212791K,AimBig Employment MILE END,MENH,7.0,19.0,5.0,29.0,1.0,11.0,0.0,...,1.0,25.0,0.0,0.0,36.842105,17.241379,9.090909,0.5,4.000000,0.5
3,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,1.0,7.0,1.0,1.0,1.0,8.0,0.0,...,0.0,0.0,0.0,0.0,14.285714,100.000000,12.500000,0.5,0.500000,0.5
4,0212793B,AimBig Employment BURLEIGH HEADS,MENH,7.0,19.0,4.0,20.0,0.0,2.0,0.0,...,0.0,24.0,0.0,0.0,36.842105,20.000000,0.000000,0.5,0.000000,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,0212902B,AimBig Employment CAMPBELLTOWN,AALL,6.0,14.0,3.0,27.0,0.0,3.0,0.0,...,2.0,26.0,0.0,0.0,42.857143,11.111111,0.000000,0.5,7.692308,0.5
179,0212902B,AimBig Employment INGLEBURN,AALL,2.0,12.0,1.0,9.0,0.0,1.0,0.0,...,0.0,8.0,0.0,0.0,16.666667,11.111111,0.000000,0.5,0.000000,0.5
180,0212904D,AimBig Employment HURSTVILLE,AALL,7.0,18.0,2.0,26.0,0.0,3.0,0.0,...,0.0,16.0,0.0,0.0,38.888889,7.692308,0.000000,0.5,0.000000,0.5
181,0212904D,AimBig Employment MIRANDA,AALL,4.0,16.0,1.0,17.0,0.0,2.0,0.0,...,0.0,16.0,0.0,0.0,25.000000,5.882353,0.000000,0.5,0.000000,0.5


In [1297]:
df_exp_per.to_csv('C:/Users/AManalo/star_ratings_new/expected_outcomes.csv')

In [1298]:
df_exp_per

,Contract_ID,Site_Name,specialist_site_type_code,Den_13_full,Den_26_full,Den_26_path,Den_26_wrkast,Den_52_full,Den_52_path,Num_13_full_exp,...,exPer_26_wrkast,exPer_52_full,exPer_52_path,SITE_CODE,52_full_avg,52_path_avg,26_wrkast_avg,26_full_avg,26_path_avg,13_full_avg
0,0212790K,AimBig Employment PROSPECT,MUSK,5.0,1.0,0.5,0.5,1.0,0.5,1.0,...,0.5,200.000000,0.5,MO87,15.7,1.1,71.9,22.8,1.2,24.0
1,0212791K,AimBig Employment MILE END,MENH,19.0,29.0,11.0,0.5,25.0,0.5,5.0,...,0.5,28.000000,0.5,MO77,15.7,1.1,71.9,22.8,1.2,24.0
2,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,7.0,1.0,8.0,0.5,0.5,0.5,0.5,...,0.5,0.500000,0.5,FW58,15.7,1.1,71.9,22.8,1.2,24.0
3,0212793B,AimBig Employment BURLEIGH HEADS,MENH,19.0,20.0,2.0,0.5,24.0,0.5,6.0,...,0.5,8.333333,0.5,N687,15.7,1.1,71.9,22.8,1.2,24.0
4,0212793B,AimBig Employment NERANG,MENH,13.0,3.0,0.5,0.5,1.0,0.5,3.0,...,0.5,200.000000,0.5,FR58,15.7,1.1,71.9,22.8,1.2,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0212902B,AimBig Employment CAMPBELLTOWN,AALL,14.0,27.0,3.0,0.5,26.0,0.5,4.0,...,0.5,15.384615,0.5,AA21,17.2,1.3,68.4,21.5,1.1,22.4
178,0212902B,AimBig Employment INGLEBURN,AALL,12.0,9.0,1.0,0.5,8.0,0.5,2.0,...,0.5,0.500000,0.5,AA11,17.2,1.3,68.4,21.5,1.1,22.4
179,0212904D,AimBig Employment HURSTVILLE,AALL,18.0,26.0,3.0,0.5,16.0,0.5,3.0,...,0.5,18.750000,0.5,A981,17.2,1.3,68.4,21.5,1.1,22.4
180,0212904D,AimBig Employment MIRANDA,AALL,16.0,17.0,2.0,0.5,16.0,0.5,2.0,...,0.5,18.750000,0.5,A991,17.2,1.3,68.4,21.5,1.1,22.4


In [1299]:
# subsetting the post-quarter actual rates for each contract
df_actPer = df_group.reset_index() # converting groupBy obj -> dataframe for subsetting
df_actPer.columns
df_actPer = df_actPer[['Contract_ID',  'Site_Name',  'specialist_site_type_code', 'actual_13_full',
       'actual_26_full', 'actual_26_path', 'actual_26_wrkast',
       'actual_52_full', 'actual_52_path']]

# subsetting the expected performance for each contract
df_exp_per = df_exp_per.reset_index()
df_exp_per.columns
df_exp_per = df_exp_per[['Contract_ID','Site_Name', 'specialist_site_type_code', 'exPer_13_full', 'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast',
       'exPer_52_full', 'exPer_52_path', '52_full_avg', '52_path_avg',  '26_wrkast_avg', '26_full_avg', '26_path_avg', '13_full_avg']]

# joining the two dataframes
df_perMea = df_actPer.merge(df_exp_per, on=['Contract_ID',  'Site_Name',  'specialist_site_type_code'], how='inner')

df_perMea.columns

df_perMea.isna().sum()
df_perMea.replace([np.inf, -np.inf], 0, inplace=True)

In [1300]:
df_perMea

,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,exPer_26_path,exPer_26_wrkast,exPer_52_full,exPer_52_path,52_full_avg,52_path_avg,26_wrkast_avg,26_full_avg,26_path_avg,13_full_avg
0,0212790K,AimBig Employment PROSPECT,MUSK,40.000000,100.000000,0.500000,0.5,100.000000,0.5,20.000000,...,0.500000,0.5,200.000000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
1,0212791K,AimBig Employment MILE END,MENH,36.842105,17.241379,9.090909,0.5,4.000000,0.5,26.315789,...,9.090909,0.5,28.000000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
2,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,14.285714,100.000000,12.500000,0.5,0.500000,0.5,0.500000,...,12.500000,0.5,0.500000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
3,0212793B,AimBig Employment BURLEIGH HEADS,MENH,36.842105,20.000000,0.000000,0.5,0.000000,0.5,31.578947,...,50.000000,0.5,8.333333,0.5,15.7,1.1,71.9,22.8,1.2,24.0
4,0212793B,AimBig Employment NERANG,MENH,38.461538,100.000000,0.500000,0.5,100.000000,0.5,23.076923,...,0.500000,0.5,200.000000,0.5,15.7,1.1,71.9,22.8,1.2,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0212902B,AimBig Employment CAMPBELLTOWN,AALL,42.857143,11.111111,0.000000,0.5,7.692308,0.5,28.571429,...,0.500000,0.5,15.384615,0.5,17.2,1.3,68.4,21.5,1.1,22.4
178,0212902B,AimBig Employment INGLEBURN,AALL,16.666667,11.111111,0.000000,0.5,0.000000,0.5,16.666667,...,100.000000,0.5,0.500000,0.5,17.2,1.3,68.4,21.5,1.1,22.4
179,0212904D,AimBig Employment HURSTVILLE,AALL,38.888889,7.692308,0.000000,0.5,0.000000,0.5,16.666667,...,0.500000,0.5,18.750000,0.5,17.2,1.3,68.4,21.5,1.1,22.4
180,0212904D,AimBig Employment MIRANDA,AALL,25.000000,5.882353,0.000000,0.5,0.000000,0.5,12.500000,...,0.500000,0.5,18.750000,0.5,17.2,1.3,68.4,21.5,1.1,22.4


### Creating the performance measure scores
 replace weightings - DMS, ESS specific 

In [1301]:
df_perMea = df_perMea.fillna(0)

df_perMea['13_full'] = df_perMea['actual_13_full'] / df_perMea['exPer_13_full']
df_perMea['26_full'] = df_perMea['actual_26_full'] / df_perMea['exPer_26_full']
df_perMea['26_path'] = df_perMea['actual_26_path'] / df_perMea['exPer_26_path']
df_perMea['26_wrkast'] = df_perMea['actual_26_wrkast'] / df_perMea['exPer_26_wrkast']
df_perMea['26_comb'] = (df_perMea['26_full'] + df_perMea['26_path'] + df_perMea['26_wrkast']).round(2)

df_perMea['52_full'] = df_perMea['actual_52_full'] / df_perMea['exPer_52_full']
df_perMea['52_path'] = df_perMea['actual_52_path'] / df_perMea['exPer_52_path']
df_perMea['52_comb'] = df_perMea['52_full'] + df_perMea['52_path']

# for the organisational averages (Sub368)
df_perMea['26_full_avg'] = df_perMea['actual_26_full']
df_perMea['26_path_avg'] = df_perMea['actual_26_path']
df_perMea['26_wrkast_avg'] = df_perMea['actual_26_wrkast']
df_perMea['26_comb_avg'] = (df_perMea['26_full_avg'] + df_perMea['26_path_avg'] + df_perMea['26_wrkast_avg']).round(2)

df_perMea['52_full_avg'] = df_perMea['actual_52_full']
df_perMea['52_path_avg'] = df_perMea['actual_52_path']
df_perMea['52_comb_avg'] = (df_perMea['52_full_avg'] + df_perMea['52_path_avg']).round()

# replace infinity -> 

df_perMea.to_csv('C:/Users/AManalo/star_ratings_new/df_test2.csv')
# replace inf with 0 

In [1302]:
df_perMea

,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,13_full,26_full,26_path,26_wrkast,26_comb,52_full,52_path,52_comb,26_comb_avg,52_comb_avg
0,0212790K,AimBig Employment PROSPECT,MUSK,40.000000,100.000000,0.500000,0.5,100.000000,0.5,20.000000,...,2.000000,1.000000,1.0,1.0,3.00,0.500000,1.0,1.500000,101.00,100.0
1,0212791K,AimBig Employment MILE END,MENH,36.842105,17.241379,9.090909,0.5,4.000000,0.5,26.315789,...,1.400000,0.714286,1.0,1.0,2.71,0.142857,1.0,1.142857,26.83,4.0
2,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,14.285714,100.000000,12.500000,0.5,0.500000,0.5,0.500000,...,28.571429,200.000000,1.0,1.0,202.00,1.000000,1.0,2.000000,113.00,1.0
3,0212793B,AimBig Employment BURLEIGH HEADS,MENH,36.842105,20.000000,0.000000,0.5,0.000000,0.5,31.578947,...,1.166667,1.333333,0.0,1.0,2.33,0.000000,1.0,1.000000,20.50,0.0
4,0212793B,AimBig Employment NERANG,MENH,38.461538,100.000000,0.500000,0.5,100.000000,0.5,23.076923,...,1.666667,1.500000,1.0,1.0,3.50,0.500000,1.0,1.500000,101.00,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0212902B,AimBig Employment CAMPBELLTOWN,AALL,42.857143,11.111111,0.000000,0.5,7.692308,0.5,28.571429,...,1.500000,1.500000,0.0,1.0,2.50,0.500000,1.0,1.500000,11.61,8.0
178,0212902B,AimBig Employment INGLEBURN,AALL,16.666667,11.111111,0.000000,0.5,0.000000,0.5,16.666667,...,1.000000,22.222222,0.0,1.0,23.22,0.000000,1.0,1.000000,11.61,0.0
179,0212904D,AimBig Employment HURSTVILLE,AALL,38.888889,7.692308,0.000000,0.5,0.000000,0.5,16.666667,...,2.333333,1.000000,0.0,1.0,2.00,0.000000,1.0,1.000000,8.19,0.0
180,0212904D,AimBig Employment MIRANDA,AALL,25.000000,5.882353,0.000000,0.5,0.000000,0.5,12.500000,...,2.000000,0.333333,0.0,1.0,1.33,0.000000,1.0,1.000000,6.38,0.0


### Step 4: Standardise the performance measure scores
Ratio of actual rate pre-standardisation and post-standardisation.

    - scale 0 to 4
    - Using MinMaxScalar
    - Only the combined outcomes - 13, 26, 52

In [1304]:
from sklearn.preprocessing import MinMaxScaler


# Function to standardize values in a row
def standardize_row(row):
    min_value = row.min()
    max_value = row.max()
    if max_value == min_value:
        return pd.Series([0 for _ in range(len(row))], index=row.index)
    else:
        return pd.Series(4 * (row - min_value) / (max_value - min_value), index=row.index)


# subset the dataframe, then rejoin -> RESET_INDEX first to transform into dataframe, then rejoin, then groupBy

included_columns = ['13_full', '26_comb', '52_comb', '13_full_avg', '26_comb_avg', '52_comb_avg']


# 1) Standardise non nat avg columns first


columns_to_standardize = [
    '13_full',
    '26_comb',
    '52_comb'
]

standardized_columns = [f'std_{col}' for col in columns_to_standardize]
df_perMea[standardized_columns] = df_perMea[columns_to_standardize].apply(standardize_row, axis=1)


# 2) Standardise nat avg columns separately

# Assuming your DataFrame is named df_perMea
# Replace these column names with the appropriate ones from your DataFrame
columns_to_standardize = [
    '13_full_avg', '26_comb_avg', '52_comb_avg'
]
standardized_columns = [f'std_{col}' for col in columns_to_standardize]
df_perMea[standardized_columns] = df_perMea[columns_to_standardize].apply(standardize_row, axis=1)


df_perMea.head(10)

,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,exPer_13_full,...,52_path,52_comb,26_comb_avg,52_comb_avg,std_13_full,std_26_comb,std_52_comb,std_13_full_avg,std_26_comb_avg,std_52_comb_avg
0,0212790K,AimBig Employment PROSPECT,MUSK,40.000000,100.000000,0.500000,0.5,100.000000,0.5,20.000000,...,1.0,1.500000,101.00,100.0,1.333333,4.00,0.0,0.000000,4.000000,3.948052
1,0212791K,AimBig Employment MILE END,MENH,36.842105,17.241379,9.090909,0.5,4.000000,0.5,26.315789,...,1.0,1.142857,26.83,4.0,0.656335,4.00,0.0,3.504161,4.000000,0.000000
2,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,14.285714,100.000000,12.500000,0.5,0.500000,0.5,0.500000,...,1.0,2.000000,113.00,1.0,0.531429,4.00,0.0,0.821429,4.000000,0.000000
3,0212793B,AimBig Employment BURLEIGH HEADS,MENH,36.842105,20.000000,0.000000,0.5,0.000000,0.5,31.578947,...,1.0,1.000000,20.50,0.0,0.501253,4.00,0.0,4.000000,3.416667,0.000000
4,0212793B,AimBig Employment NERANG,MENH,38.461538,100.000000,0.500000,0.5,100.000000,0.5,23.076923,...,1.0,1.500000,101.00,100.0,0.333333,4.00,0.0,0.000000,4.000000,3.948052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0212902B,AimBig Employment CAMPBELLTOWN,AALL,42.857143,11.111111,0.000000,0.5,7.692308,0.5,28.571429,...,1.0,1.500000,11.61,8.0,0.000000,4.00,0.0,4.000000,1.002778,0.000000
178,0212902B,AimBig Employment INGLEBURN,AALL,16.666667,11.111111,0.000000,0.5,0.000000,0.5,16.666667,...,1.0,1.000000,11.61,0.0,0.000000,4.00,0.0,4.000000,2.073214,0.000000
179,0212904D,AimBig Employment HURSTVILLE,AALL,38.888889,7.692308,0.000000,0.5,0.000000,0.5,16.666667,...,1.0,1.000000,8.19,0.0,4.000000,3.00,0.0,4.000000,1.462500,0.000000
180,0212904D,AimBig Employment MIRANDA,AALL,25.000000,5.882353,0.000000,0.5,0.000000,0.5,12.500000,...,1.0,1.000000,6.38,0.0,4.000000,1.32,0.0,4.000000,1.139286,0.000000


In [1305]:
df_perMea.columns

df_perMea = df_perMea.drop(columns=['13_full', '26_comb', '52_comb', '13_full_avg', '26_comb_avg', '52_comb_avg'])
df_perMea = df_perMea.rename(columns={ 'std_13_full' : '13_full', 'std_26_comb' : '26_comb', 'std_52_comb' : '52_comb', 'std_13_full_avg' : '13_full_avg', 'std_26_comb_avg' : '26_comb_avg', 'std_52_comb_avg' : '52_comb_avg'})

### Step 5: Calculate the Overall Performance Score
- Multiplying the standardised performance scores with the DMS/ESS weighting
TO-DO: multiply with the respective DMS/ESS weight

In [1307]:
df_perMea.to_csv('C:/Users/AManalo/star_ratings_new/df_perMea.csv')

In [1308]:
df_perMea['13_full'] = df_perMea['13_full'] * 0.228
df_perMea['26_comb'] = df_perMea['26_comb'] * 0.457
df_perMea['52_comb'] = df_perMea['52_comb'] * 0.285

df_perMea['overall_perScore'] = df_perMea['13_full'] + df_perMea['26_comb'] + df_perMea['52_comb']


#avg performance score -> against national average
df_perMea['13_full_avg'] = df_perMea['13_full_avg'] * 0.228
df_perMea['26_comb_avg'] = df_perMea['26_comb_avg'] * 0.457
df_perMea['52_comb_avg'] = df_perMea['52_comb_avg'] * 0.285

df_perMea['overall_perScore_avg'] = df_perMea['13_full_avg'] + df_perMea['26_comb_avg'] + df_perMea['52_comb_avg']



### Step 6: Calculate the Star Rating Percentage
- Calculated using the formula:
str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 


str% = (performance score for contract X) - (National Average contract perf score)/(National Average contract perf score) * 100 

In [1309]:
df_perMea = df_perMea.reset_index()
df_perMea.columns

Index(['index', 'Contract_ID', 'Site_Name', 'specialist_site_type_code',
       'actual_13_full', 'actual_26_full', 'actual_26_path',
       'actual_26_wrkast', 'actual_52_full', 'actual_52_path', 'exPer_13_full',
       'exPer_26_full', 'exPer_26_path', 'exPer_26_wrkast', 'exPer_52_full',
       'exPer_52_path', '52_full_avg', '52_path_avg', '26_wrkast_avg',
       '26_full_avg', '26_path_avg', '26_full', '26_path', '26_wrkast',
       '52_full', '52_path', '13_full', '26_comb', '52_comb', '13_full_avg',
       '26_comb_avg', '52_comb_avg', 'overall_perScore',
       'overall_perScore_avg'],
      dtype='object')

In [1311]:
# calculating the star rating percentage
df_perMea['star_rating_perc'] = ((df_perMea['overall_perScore'] - df_perMea['overall_perScore_avg'])/ (df_perMea['overall_perScore_avg'])) * 100

In [1312]:
df_perMea

,index,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,...,52_path,13_full,26_comb,52_comb,13_full_avg,26_comb_avg,52_comb_avg,overall_perScore,overall_perScore_avg,star_rating_perc
0,0,0212790K,AimBig Employment PROSPECT,MUSK,40.000000,100.000000,0.500000,0.5,100.000000,0.5,...,1.0,0.304000,1.82800,0.0,0.000000,1.828000,1.125195,2.132000,2.953195,-27.806997
1,1,0212791K,AimBig Employment MILE END,MENH,36.842105,17.241379,9.090909,0.5,4.000000,0.5,...,1.0,0.149644,1.82800,0.0,0.798949,1.828000,0.000000,1.977644,2.626949,-24.717051
2,2,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,14.285714,100.000000,12.500000,0.5,0.500000,0.5,...,1.0,0.121166,1.82800,0.0,0.187286,1.828000,0.000000,1.949166,2.015286,-3.280924
3,3,0212793B,AimBig Employment BURLEIGH HEADS,MENH,36.842105,20.000000,0.000000,0.5,0.000000,0.5,...,1.0,0.114286,1.82800,0.0,0.912000,1.561417,0.000000,1.942286,2.473417,-21.473574
4,4,0212793B,AimBig Employment NERANG,MENH,38.461538,100.000000,0.500000,0.5,100.000000,0.5,...,1.0,0.076000,1.82800,0.0,0.000000,1.828000,1.125195,1.904000,2.953195,-35.527450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,177,0212902B,AimBig Employment CAMPBELLTOWN,AALL,42.857143,11.111111,0.000000,0.5,7.692308,0.5,...,1.0,0.000000,1.82800,0.0,0.912000,0.458269,0.000000,1.828000,1.370269,33.404420
178,178,0212902B,AimBig Employment INGLEBURN,AALL,16.666667,11.111111,0.000000,0.5,0.000000,0.5,...,1.0,0.000000,1.82800,0.0,0.912000,0.947459,0.000000,1.828000,1.859459,-1.691832
179,179,0212904D,AimBig Employment HURSTVILLE,AALL,38.888889,7.692308,0.000000,0.5,0.000000,0.5,...,1.0,0.912000,1.37100,0.0,0.912000,0.668362,0.000000,2.283000,1.580363,44.460527
180,180,0212904D,AimBig Employment MIRANDA,AALL,25.000000,5.882353,0.000000,0.5,0.000000,0.5,...,1.0,0.912000,0.60324,0.0,0.912000,0.520654,0.000000,1.515240,1.432654,5.764578


Converting the star rating percentage -> 1-5 star scale

In [1313]:
def convert_percentage_to_star_rating(percentage):
    if percentage >= 40:
        return 5
    elif 20 <= percentage <= 39:
        return 4
    elif -19 <= percentage <= 19:
        return 3
    elif -49 <= percentage <= -20:
        return 2
    elif percentage <= -50:
        return 1

In [1314]:
df_perMea['star_rating'] = df_perMea['star_rating_perc'].apply(convert_percentage_to_star_rating)

In [1315]:
df_perMea

,index,Contract_ID,Site_Name,specialist_site_type_code,actual_13_full,actual_26_full,actual_26_path,actual_26_wrkast,actual_52_full,actual_52_path,...,13_full,26_comb,52_comb,13_full_avg,26_comb_avg,52_comb_avg,overall_perScore,overall_perScore_avg,star_rating_perc,star_rating
0,0,0212790K,AimBig Employment PROSPECT,MUSK,40.000000,100.000000,0.500000,0.5,100.000000,0.5,...,0.304000,1.82800,0.0,0.000000,1.828000,1.125195,2.132000,2.953195,-27.806997,2.0
1,1,0212791K,AimBig Employment MILE END,MENH,36.842105,17.241379,9.090909,0.5,4.000000,0.5,...,0.149644,1.82800,0.0,0.798949,1.828000,0.000000,1.977644,2.626949,-24.717051,2.0
2,2,0212791K,AimBig Employment Pty Ltd. SEATON,MENH,14.285714,100.000000,12.500000,0.5,0.500000,0.5,...,0.121166,1.82800,0.0,0.187286,1.828000,0.000000,1.949166,2.015286,-3.280924,3.0
3,3,0212793B,AimBig Employment BURLEIGH HEADS,MENH,36.842105,20.000000,0.000000,0.5,0.000000,0.5,...,0.114286,1.82800,0.0,0.912000,1.561417,0.000000,1.942286,2.473417,-21.473574,2.0
4,4,0212793B,AimBig Employment NERANG,MENH,38.461538,100.000000,0.500000,0.5,100.000000,0.5,...,0.076000,1.82800,0.0,0.000000,1.828000,1.125195,1.904000,2.953195,-35.527450,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,177,0212902B,AimBig Employment CAMPBELLTOWN,AALL,42.857143,11.111111,0.000000,0.5,7.692308,0.5,...,0.000000,1.82800,0.0,0.912000,0.458269,0.000000,1.828000,1.370269,33.404420,4.0
178,178,0212902B,AimBig Employment INGLEBURN,AALL,16.666667,11.111111,0.000000,0.5,0.000000,0.5,...,0.000000,1.82800,0.0,0.912000,0.947459,0.000000,1.828000,1.859459,-1.691832,3.0
179,179,0212904D,AimBig Employment HURSTVILLE,AALL,38.888889,7.692308,0.000000,0.5,0.000000,0.5,...,0.912000,1.37100,0.0,0.912000,0.668362,0.000000,2.283000,1.580363,44.460527,5.0
180,180,0212904D,AimBig Employment MIRANDA,AALL,25.000000,5.882353,0.000000,0.5,0.000000,0.5,...,0.912000,0.60324,0.0,0.912000,0.520654,0.000000,1.515240,1.432654,5.764578,3.0


In [1316]:
df_perMea.to_csv('C:/Users/AManalo/star_ratings_new/star_ratings.csv')